Description of constraints and variables..

$c_{i,j,k,l}$ - действие взятия кубика $j$-го цвета $i$-ым манипулятором из $k$-ой кучки $l$-ым номером. ($l$-номером - в смысле, кубик, который этот манипулятор взял первым - имеет первый номер и тд).

Тогда:

$c_{ijkl} \in \{0,1\}$

$c_{ijkl} \geq 0, c_{ijkl} \leq 1$

**DONE**

$\forall i,j,k, ~ l_2 \geq l_1 \Rightarrow c_{ijk,l_2} \geq c_{ijk,l_1}$ - не можем взять второй кубик в манипулятро, пока не взяли первый и тд.

FIXED:

$  l_2 \geq l_1 \Rightarrow \sum_{j,k} c_{ijk,l_2} \geq  \sum_{j,k} c_{ijk,l_1}$

TEMPORARY: just tower height = 5 constraint **DONE**

$\forall j,k \sum_{i,l} c_{ijkl} \leq 1$ - берем каждый кубик опр.цвета $j$ из опр. кучки $k$ не более 1го раза

$\forall i,l \sum_{j,k} c_{ijkl} \leq 1$ - в каждом манипуляторе $i$ на опр. высоте $l$ не более 1го кубика

TEMPORARY : not $\leq$ but $=$ because fixed picking 15 cubes. **DONE**

Введем $b_{ijl} = \sum_k c_{ijkl}$ - наличие кубика цвета $j$ в манипуляторе $i$ на высоте $l$ (уже не важно из какой кучки). Для этой переменной можно записать условия на бонус очков в каждом манипуляторе:

Если $a$, $b$, $c$ - бонусная последовательность цветов, то ее можно учесть след. образом для каждого манипулятора.
Пусть $d_i$ - действие выкладывания $i$ манипулятора. Тогда начисление бонусных баллов можно представить в виде переменной $\xi_i$:

$\xi_i \geq 0, \xi_i \leq 1$

У нас есть 6 возможных мест для последовательности из трех кубиков правильного цвета:
наличие на $j$ из 6 мест бонусной посл. в $i$ манипуляторе обозначим за $\pi_{ij}$. Тогда

$\pi_{ij} \geq 0$

$\pi_{i,0} \leq b_{i,0,a}$

$\pi_{i,0} \leq b_{i,0,a}$

$\pi_{i,0} \leq b_{i,1,b}$

$\pi_{i,0} \leq b_{i,2,c}$

и так далее для $\pi_{i,1..}$


Двух бонусных последовательностей в одном манипуляторе быть не может, поэтому
$\xi_i \leq \sum_j \pi_{i,j}$ and $\forall\; i\; \xi_i \in \{0, 1\}$

Соответственно, если есть бонусная посл., то $\xi_i \in \{0,1\}$, а если ее нет, то $\xi_i = 0$. Поэтому, если добавить слагаемое $+A \xi_i$, то $\max...$ будет выбирать его $1$ при возможности.


А для учитывания уже конкретно наличия бонуса в 30 очков остается проверить, выложили ли мы эту башенку:

$\zeta_i \leq \xi_i, \zeta_i \leq d_i, \zeta_i \geq 0$

**obj $ .. + 30\sum_i \zeta_i$ **

Чтобы учесть взятие кубиков с разных сторон, на самом деле, у нас будут переменные $p_{ijklm}$. Здесь $m$ отвечает за то, с какой из 4х сторон мы подъехали к кубику $i$ манипулятором. Не всегда все из этих подъездов возможны. Пусть кубик цвета $j_2$  блокируют подъезд к кубику $j_1$ со стороны $m = 0$. Тогда, 
$index(p_{i,j_1,klm}) \geq \sum_{i,m,l} index(p_{i,j_2,klm})$
Для других цветов пар $j_1$,$j_2$ и подъездов $m$ необходимо написать такие же условия, если блокирует.


Основная переменная - матрица $x_{ij}$.  $x_{ij} = 1$, если j-е действие ($a_j$) является i-ым в порядке выполнения.

$index(a_j) = (1,2,3...)^\top x_{ij}$ 
-дает порядковый номер действия $a_j$, если оно включено в выбранную последовательность. Иначе 0.

Время. Пусть мы тратим на действия $\omega_i$ времени, а на переезды между действиями $\Omega_{ij}$.


$T_1 = \sum_{ij} \sum_N x_{N,i} x_{N-1,j} \Omega_{ij} $,

$T_2 = \omega_j \sum_i x_{ij}$


$T = T_1 + T_2$

objective - формируется за собранные кубики. За каждый кубик дается равно его высоте количество очков, начинается с 1. (+бонус). Запишем в $t_i$ очки за каждую башню:

$t_i = \sum_{j,i} b_{i,j,l}$ ... Но пока $t_i = 5$. И набираемые очки за это фиксированы, максимизировать их не нужно.

$\max_i \sum_i \zeta_i$

бонус +

брать кубики с разных сторон +

выкладываться + 

выкладываться в разных точках ?

разная высота башен -


In [14]:
import numpy as np
import scipy as sp
import scipy.optimize
import cvxpy as cvx
from numpy.random import randn
import matplotlib.pyplot as plt
%matplotlib inline

In [63]:
import gurobipy
import functools

In [137]:
I, J, K, L, M = 3, 5, 6, 5, 4
pick_action_dims = np.array([I,J,K,L,M])
N = np.prod(pick_action_dims)
print(N)
# 3 manipulators
# 5 colors (and cubes on each pickpoint)
# 6 pickpoints
# 5 places in each manipulator
# 4 directions from which each cube can be picked up

1800


In [138]:
# having for simplicity constraint to load full manipulators, lets also assume we unload once
# so we have N different actions of picking and 1 to unload towers.
# also fake action to with start point: total N+2
# we need to choose 15 picks + 1 fake start + 1 fake unload = 17
S = 17

In [139]:
T = 100 #match duration

In [140]:
# picking actions p_i,j,k,l,m where 
# i for manipulator
# j for color
# k for pickpoint
# l for order of picking to each manipulator (first picked - l = 0, second -  l = 1...)
# m for direction..
def orderer(*args):
    order = [1]
    for a in list(reversed(args))[:-1]:
        order.append(order[-1]*a)
    return list(reversed(order))
pick = orderer(I,J,K,L,M)
def action_index(order, *args): 
    return np.sum(order * np.array(args))

In [141]:
action_index(pick,0,0,1,1,0)

24

In [142]:
x = cvx.Variable(S,N+2)

In [231]:
#fake constraint about start

def start_var_constraints():
    return [x[0,-2] == 1]

In [232]:
constraints = start_var_constraints()

In [233]:
p = cvx.Variable(N)

In [234]:
def p_var_constraints():
    return [p.T == cvx.sum_entries(x,axis=0)[:-2]]
        

In [235]:
constraints += p_var_constraints()
constraints

[EqConstraint(Expression(AFFINE, UNKNOWN, (1, 1)), Constant(CONSTANT, POSITIVE, (1, 1))),
 EqConstraint(Expression(AFFINE, UNKNOWN, (1, 1800)), Expression(AFFINE, UNKNOWN, (1, 1800)))]

$c_{i,j,k,l}$ - действие взятия кубика $j$-го цвета $i$-ым манипулятором из $k$-ой кучки $l$-ым номером. ($l$-номером - в смысле, кубик, который этот манипулятор взял первым - имеет первый номер и тд).

Тогда:

$c_{ijkl} \in \{0,1\}$

$c_{ijkl} \geq 0, c_{ijkl} \leq 1$

In [236]:
# for many constraints we don't care from which direction we pick cube
# and order of action execution
c = cvx.Variable(I*J*K*L)
c_order = orderer(I,J,K,L)

In [237]:
max_indexes = np.array([I,J,K,L,M])
i_, j_, k_, l_, m_ = 0, 1 ,2 ,3, 4
def matrix_for_constraint(indexes1, indexes2):
    D1 = np.prod(max_indexes[indexes1])
    D2 = np.prod(max_indexes[indexes2])
    c_mat = np.zeros((D1,D2))
    order1 = orderer(*max_indexes[indexes1])
    order2 = orderer(*max_indexes[indexes2])
    for i in range(I):
        for j in range(J):
            for k in range(K):
                for l in range(L):
                    for m in range(M):
                        ids = np.array([i,j,k,l,m])
                        c_mat[action_index(order1,*(ids[indexes1]))][action_index(order2,*(ids[indexes2]))] = 1
    return c_mat

In [238]:
def c_var_constraints():
    #c_mat = np.zeros((I*J*K*L,N))
    #for i in range(I):
    #    for j in range(J):
    #        for k in range(K):
    #            for l in range(L):
    #                for m in range(M):
    #                    c_mat[action_index(c_order,i,j,k,l)][action_index(pick,i,j,k,l,m)] = 1
    c_mat = matrix_for_constraint([i_,j_,k_,l_],[i_,j_,k_,l_,m_])
    cnts = [c == c_mat*p, c >= 0, c <= 1] #basic
    return cnts


In [239]:
constraints += c_var_constraints()
constraints

[EqConstraint(Expression(AFFINE, UNKNOWN, (1, 1)), Constant(CONSTANT, POSITIVE, (1, 1))),
 EqConstraint(Expression(AFFINE, UNKNOWN, (1, 1800)), Expression(AFFINE, UNKNOWN, (1, 1800))),
 EqConstraint(Variable(450, 1), Expression(AFFINE, UNKNOWN, (450, 1))),
 LeqConstraint(Constant(CONSTANT, ZERO, (1, 1)), Variable(450, 1)),
 LeqConstraint(Variable(450, 1), Constant(CONSTANT, POSITIVE, (1, 1)))]

$\forall i,j,k, ~ l_2 \geq l_1 \Rightarrow c_{ijk,l_2} \geq c_{ijk,l_1}$ - не можем взять второй кубик в манипулятро, пока не взяли первый и тд.

FIXED:

$  l_2 \geq l_1 \Rightarrow \sum_{j,k} c_{ijk,l_2} \geq  \sum_{j,k} c_{ijk,l_1}$

TEMPORARY: just tower height = 5 constraint **DONE**

In [240]:
#t = cvx.Variable(I*L)
t_order = orderer(I)
def tower_height_contstraints():
    #tower_height_matrix = np.zeros((I,I*J*K*L))
    # matrix for l2>l1 => \sum_j,k c_i,j,k,l2 >= \sum_j,k c_i,j,k,l1 (tower order check)
    # but now just tower height == 5 
                                  
    #for i in range(I):
    #    for j in range(J):
    #        for k in range(K):
    #            for l in range(L):
    #                tower_height_matrix[action_index(t_order,i)][action_index(c_order,i,j,k,l)] = 1
                    
                    
    tower_height_matrix = matrix_for_constraint([i_],[i_,j_,k_,l_])
    return [tower_height_matrix*c == 5]

In [241]:
constraints += tower_height_contstraints()
print(tower_height_contstraints())

[EqConstraint(Expression(AFFINE, UNKNOWN, (3, 1)), Constant(CONSTANT, POSITIVE, (1, 1)))]


$\forall j,k \sum_{i,l} c_{ijkl} \leq 1$ - берем каждый кубик опр.цвета $j$ из опр. кучки $k$ не более 1го раза

$\forall i,l \sum_{j,k} c_{ijkl} \leq 1$ - в каждом манипуляторе $i$ на опр. высоте $l$ не более 1го кубика

TEMPORARY : equalities instead of inequalities

In [242]:
def unique_color_pickpoint_constraint():
    c_p_mat = matrix_for_constraint([i_,l_],[i_,j_,k_,l_])
    c_m_mat = matrix_for_constraint([j_,k_],[i_,j_,k_,l_])
    return [c_p_mat*c == 1, c_m_mat*c == 1]

In [243]:
constraints += unique_color_pickpoint_constraint()

Введем $b_{ijl} = \sum_k c_{ijkl}$ - наличие кубика цвета $j$ в манипуляторе $i$ на высоте $l$ (уже не важно из какой кучки). Для этой переменной можно записать условия на бонус очков в каждом манипуляторе:

In [244]:
b = cvx.Variable(I*J*L)

In [245]:
def b_var_constraints():
    b_mat = matrix_for_constraint([i_,j_,l_],[i_,j_,k_,l_])
    return [b == b_mat*c]

In [246]:
constraints += b_var_constraints()

Если $a$, $b$, $c$ - бонусная последовательность цветов, то ее можно учесть след. образом для каждого манипулятора.
Пусть $d_i$ - действие выкладывания $i$ манипулятора. Тогда начисление бонусных баллов можно представить в виде переменной $\xi_i$:

$\xi_i \geq 0, \xi_i \leq 1$

У нас есть 6 возможных мест для последовательности из трех кубиков правильного цвета:
наличие на $j$ из 6 мест бонусной посл. в $i$ манипуляторе обозначим за $\pi_{ij}$. Тогда

$\pi_{ij} \geq 0$

$\pi_{i,0} \leq b_{i,0,a}$

$\pi_{i,0} \leq b_{i,1,b}$

$\pi_{i,0} \leq b_{i,2,c}$

и так далее для $\pi_{i,1..}$


In [247]:
bs = [0,1,2] # bonus sequence; to be recognised via CV algorithms on the start of match

In [248]:
#6 because 6 possible bonus subsequences of 3 cubes in 5 cubes 
PS = 6
π = cvx.Variable(I,PS)
possible_subsequences = [[0,1,2],[1,2,3],[2,3,4],[4,3,2],[3,2,1],[2,1,0]]
def π_var_constraints():
    cnts = [π >= 0]
    b_order = orderer(I,J,L)
    for i in range(I):
        for num_ps, ps in enumerate(possible_subsequences):
            for num_bonus_color,bonus_color in enumerate(bs):
                cnts.append(π[i,num_ps] <= b[action_index(b_order,i,ps[num_bonus_color],bonus_color)])
    return cnts

За две бонусные последовательности в манипуляторе лишних очков не дают, поэтому
$\xi_i \leq \sum_j \pi_{i,j}$ and $\forall\; i\; \xi_i \in \{0, 1\}$

Соответственно, если есть бонусная посл., то $\xi_i \in \{0,1\}$, а если ее нет, то $\xi_i = 0$. Поэтому, если добавить слагаемое $+A \xi_i$, то $\max...$ будет выбирать его $1$ при возможности.


In [249]:
ξ = cvx.Variable(I)

In [250]:
def ξ_var_constraints():
    return [ξ >=0, ξ <= 1, ξ <= cvx.sum_entries(π,axis=1)]
    

In [251]:
constraints += ξ_var_constraints()

А для учитывания уже конкретно наличия бонуса в 30 очков остается проверить, выложили ли мы эту башенку:

$\zeta_i \leq \xi_i, \zeta_i \leq d_i, \zeta_i \geq 0$

**obj $ .. + 30\sum_i \zeta_i$ **

TEMPORARY : check for all towers at once, not $d_i$, but $d$

In [252]:
ζ = cvx.Variable(I)
d = cvx.Variable() # d - last action in actions list
def ζ_d_var_constraints():
    return [ζ <= ξ, ζ <= d, ζ >= 0, d == cvx.sum_entries(x,axis=0)[-1]]

In [253]:
constraints += ζ_d_var_constraints()

In [254]:
objective = cvx.Maximize(cvx.sum_entries(ζ))

Чтобы учесть взятие кубиков с разных сторон, на самом деле, у нас будут переменные $p_{ijklm}$. Здесь $m$ отвечает за то, с какой из 4х сторон мы подъехали к кубику $i$ манипулятором. Не всегда все из этих подъездов возможны. Пусть кубик цвета $j_2$  блокируют подъезд к кубику $j_1$ со стороны $m = 0$. Тогда, 
$index(p_{i,j_1,klm}) \geq \sum_{i,m,l} index(p_{i,j_2,klm})$
Для других цветов пар $j_1$,$j_2$ и подъездов $m$ необходимо написать такие же условия, если блокирует.


Основная переменная - матрица $x_{ij}$.  $x_{ij} = 1$, если j-е действие ($a_j$) является i-ым в порядке выполнения.

$index(a_j) = (1,2,3...)^\top x_{ij}$ 
-дает порядковый номер действия $a_j$, если оно включено в выбранную последовательность. Иначе 0.

In [259]:
index_vector = np.array(list(range(S)))+1

In [260]:
index_vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [263]:
index = cvx.Variable(N+2)
def index_var_constraints():
    return [index.T == index_vector.T*x]

In [264]:
constraints += index_var_constraints()

In [287]:
#now we need to find all blocking conditions of (m,j1,i,j2)
#firstly let's build a matrix of relations between (j2,j2) 
#  2 
#3 4 1
#  0  
# e.g. 0 is dx = 0, dy = -1 to 4

j_rel = np.zeros((J+1,J+1,2))
j_rel[0,4,0] = 0
j_rel[0,4,1] = -1
j_rel[4,2] = j_rel[0,4]
j_rel[4,0] = -j_rel[0,4]
j_rel[2,4] = -j_rel[4,2]

j_rel[3,4,0] = 1
j_rel[3,4,1] = 0
j_rel[4,3] = -j_rel[3,4]
j_rel[4,1] = j_rel[3,4]
j_rel[1,4] = -j_rel[4,1]

j_map = np.array([
                 [-1,-1,-1,-1,-1],
                 [-1,-1, 2,-1,-1],
                 [-1, 3, 4, 1,-1],
                 [-1,-1, 0,-1,-1],
                 [-1,-1,-1,-1,-1]
                 ])

for i1 in range(j_map.shape[0]):
    for j1 in range(j_map.shape[1]):
        for i2 in range(j_map.shape[0]):
            for j2 in range(j_map.shape[1]):
                if j_map[i1,j1] >= 0 and j_map[i2,j2] >= 0:
                    if i1 != i2 or j1 != j2:
                        j_rel[j_map[i1,j1],j_map[i2,j2]] = np.array([j2-j1,i2-i1])
    

In [288]:
j_rel[3,4]

array([ 1.,  0.])

In [297]:
# we need just to have free space on 3 'kletka' around the manipulator
def m_manipulator(i,m):
    return (m+i-1)%4

only_non_free_places = [np.array([0,-1]),np.array([-1,0]),np.array([0,1]),np.array([1,0])]

def only_non_free_dif(i,m):
    direction =  m_manipulator(i,m)
    return only_non_free_places(direction)



In [298]:
def j1j2_constraints():
    cnts = []
    reduce_l = matrix_for_constraint([i_,j_,k_,m_],[i_,j_,k_,l_,m_])
    for k in range(K):
        for i in range(I):
            

SyntaxError: unexpected EOF while parsing (<ipython-input-298-c768089daf07>, line 6)